# Running with `xyz` Grain Boundary Files

The original `gblearn` package was designed to work with LAMMPS dump files. However, `xyz` files are good format to support as well. This notebook tests the implementation of the `xyz` parser/converter for GBs.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from gblearn.xyz import XYZParser
from gblearn.gb import GrainBoundaryCollection as GBC
from gblearn.gb import GrainBoundaryCollection as GBC

In [ ]:
im = GBC("imeall", "/Users/trunks/codes/gblearn-dist/tests/xyz/", 
         "/Users/trunks/temp/gblearn/", 
         r"^(?P<gbid>\d+)_Dud.xyz$",
         rcut=3.25, lmax=12, nmax=12, sigma=0.5)

In [ ]:
im.load(XYZParser, Z=26, method="cna_z", pattr="cna", cna_val=3)

In [ ]:
im.soap()

In [ ]:
import quippy

In [ ]:
seedgb = quippy.Atoms("/Users/trunks/codes/gblearn-dist/tests/xyz/00110391110_Dud.xyz")

In [ ]:
import numpy as np
positions = np.array(seedgb.pos.T)

In [ ]:
zm = np.logical_and(positions[:,2] > 55, positions[:,2] < 65)
ym = np.logical_and(positions[:,1] > 25, positions[:,1] < 35)
cm = np.logical_and(zm, ym)

In [ ]:
indices = np.where(cm)

In [ ]:
subseed = seedgb[indices]

In [ ]:
subseed.write("/Users/trunks/temp/seed.xyz")

In [ ]:
selected = positions[cm,:]

In [ ]:
xl, yl, zl = [(np.min(selected[:,i]), np.max(selected[:,i])) for i in range(3)]

In [ ]:
xl[1]-16.98

In [ ]:
subseed.positions[:,1] -= yl[0]
subseed.positions[:,2] -= zl[0]

In [ ]:
subseed.lattice

In [ ]:
subseed.lattice = np.array([[ 16.98      ,   0.        ,   0.        ],
                            [0,yl[1]-yl[0],0],
                            [0,0,zl[1]-zl[0]]])

In [ ]:
from gblearn.soap import SOAPCalculator
calc = SOAPCalculator(rcut=3.25)
Pseed = calc.calc(subseed, 26)["descriptor"]

In [ ]:
positions = subseed.positions
zmi = np.logical_and(positions[:,2] > 59-zl[0], positions[:,2] < 61-zl[0])
ymi = np.logical_and(positions[:,1] > 29-yl[0], positions[:,1] < 31-yl[0])
cmi = np.logical_and(zmi, ymi)

In [ ]:
seedsoap = np.where(cmi)[0]
seedsoap

In [ ]:
Pi = Pseed[seedsoap,:]

In [ ]:
[np.max(np.abs(Pi[0,:]-Pi[i,:])) for i in range(1, len(Pi))]

In [ ]:
alphafe = Pi[0,:]

In [ ]:
np.save("/Users/trunks/codes/gblearn-dist/tests/xyz/alpha_fe.npy", alphafe)